In [47]:
#this will import all the following libraries that we need to run the project below
from newsapi import NewsApiClient
import smtplib 
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
newsapi = NewsApiClient(api_key="")

#status 
#totalResults
#articles

In [48]:
"""
The following method is used to get the information using the newsapi-python library.
We take in four parameters, the type of information we want, a keyword, and country and then the api itself,
inside the method depending on the type the user chooses whether or not they want the top headlines or articles. 

If the user wants the top headlines then we use the .get_top_headlines() function from the api which takes in a country as its
parameter, if the user wants to get articles then we use the .get_everything() function from the api which takes in a keyword as 
its parameter.

we then store the information in a database that is a dictionary, where the key is the article title followed by a 
list that contains the description, url, and urlToImage and we return it at the end

"""
def catchInformation(typ, keyword, cont, api):
    article_dtb = {}
    if typ.lower() == "top headlines" or typ == "topheadlines":
        top_headlines = api.get_top_headlines(country=cont)
        articles_info = top_headlines.get("articles")
        for info in articles_info:
            article_details = info
            article_dtb[str(article_details.get("title"))] = [str(article_details.get("description")),str(article_details.get("url")),str(article_details.get("urlToImage"))]
    elif typ.lower() == "get articles" or typ == "getarticles":
        get_articles = api.get_everything(q=keyword,page=1,sort_by="relevancy")
        articles_info = get_articles.get("articles")
        for info in articles_info:
            article_details = info
            article_dtb[str(article_details.get("title"))] = [str(article_details.get("description")),str(article_details.get("url")),str(article_details.get("urlToImage"))]
            
    return article_dtb

#catchInformation("getarticles","coronavirus","us",newsapi)

In [49]:
"""
This method creates the head of the email essentially, given the choice, keyword and country 
we are able to determine what the choice of the user was and then use either the keyword or country to create
the string that will be the head of the email and return it 

"""
def createHeadEmail(choice, keyword, country):
    #still have to write
    result = """"""
    if choice == "top headlines" or choice == "topheadlines":
        result = result + """Here are the top headlines for your selected country of: """ + country.upper() 
    elif choice == "get articles" or choice == "get articles":
        result = result + """Here are the most relevant articles for the keyword: """ + keyword.upper() 
        
    return result 
    

In [50]:
"""
Through the use of multi line strings we are able to construct the html code that will create the media objects for 
the body of the email. We use css classes from Bootstrap to provide some css to our code rather than it just being plain 
html. 

We have the database paramter which we use to gather the information such as the title, description and url to the article
we then construct the html and using a clickable link on the title that would re direct the user to the webpage of that given article

"""
def createHTML(database):
    #make the title a hyperlink somehow
    result = """"""
    for article in database:
        title = article
        description = database[article][0]
        url = database[article][1]
        result = result + """<div class="media">"""+ """<div class="media-body">""" + """<h5 class="mt-0">""" + """<a href=""" + url + """/> """ + title + """</a>""" + """</h5> <p class="mb-0"> """ + description + """</p> </div>"""+"""</div>"""
    return result 

#createHTML(catchInformation("top headlines", "", "us", newsapi))

In [51]:
"""
This is the main part of this project, this function allows us to send emails to the user with the information they selected
We have a html_body, email, country, keyword, and choice parameter 

Using the MIMEMultipart python library we are able to construct a message, subject and from and to person in which our email is
sent to. 

Using multi line strings we then construct a general html file which will be embedded within our email to display our data to
our user.

Using the smtplib library we are able to then to send the email through a gmail server using a personalized gmail account that was
made. 

"""
def sendEmail(html_body, email, country, keyword, choice):
    msg = MIMEMultipart("alternative")
    subject_line = ""
    if choice == "top headlines" or choice == "topheadlines":
        subject_line = "Top Headlines for: " + country.upper() 
    else:
        subject_line = "Most Relevant Articles for: " + keyword.upper()
    
    msg["Subject"] = subject_line
    msg["From"] = "email.notifications.py@gmail.com"
    msg["To"] = email
    html = """ 
    <html> 
        <head> 
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" crossorigin="anonymous">
        """ + """<h1 class="display-4"> """ + createHeadEmail(choice,keyword,country) + """ </h1> 
       </head> 
        <body> """ + html_body +  """</body> 
    </html>"""
    part2 = MIMEText(html,"html")
    
    msg.attach(part2)
    
    with smtplib.SMTP("smtp.gmail.com",587) as smtp:
        smtp.ehlo()
        smtp.starttls()
        smtp.ehlo()
        smtp.login("email.notifications.py@gmail.com","")
        smtp.sendmail("email.notifications.py@gmail.com",email,msg.as_string())
        smtp.quit()

In [53]:
"""
This handles the main logic of what the user interacts with
We ask the user for their input in which we then call the functions necessary that will handle the main portions 
of our goal. 

"""

def main():
    print("Welcome! This is the news application, where you can grab")
    print("the top news headline for a given country or search for the")
    print("most revelant articles given a keyword phrase. This information will then")
    print("be sent to your email!")
    choice = input("Please enter, top headlines or get articles: ")
    if choice == "top headlines":
        count = input("Please choose from the following countries!" + "\n" + 
                       "ae, ar, at, au, be, bg, ca, ch, cn, co, cu, cz, de, eg"+"\n"+
                       "fr, gb, gr, hk, hu, id, ie, il, in, it, jp, kr, lt, lv, ma, mx, my, ng, nl, no, nz"+"\n"+
                       "ph, pl, pt, ro, rs, ru, sa, se, sg, si, sk, th, tr, tw, ua, us, ve, za")
        data = catchInformation(choice,"",count,newsapi)
        get_html_body = createHTML(data)
        email_addr = input("Enter the email address you want this sent to!: ")
        sendEmail(get_html_body, email_addr,count, "",choice)
        print("Check your email!")
    elif choice == "get articles":
        keyword = input("Enter a keyword phrase to search for your articles!: ")
        data = catchInformation(choice,keyword,"",newsapi)
        get_html_body = createHTML(data)
        email_addr = input("Enter the email address you want this sent to!: ")
        sendEmail(get_html_body, email_addr,"", keyword,choice)
        print("Check your email!")
    else: 
        print("That is not a valid input")

#main()

In [ ]:
"""
CHANGES TO COME:

- fix some of the formatting in the html created so it looks more visually appealing to the user 
- fix some of the code to make it look more clear 

NOTES ON UPDATE 2  SUBMISSION:
api key and email password omitted since it is posted on a public repo
"""